In [6]:
import numpy as np
import pandas as pd
import os
import requests
from tqdm import tqdm
import time as tm
import time

In [7]:
os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 

In [8]:
df = pd.read_csv('Cluster/df_15c.csv')

In [9]:
puertos = pd.read_csv('Bases/puertos.csv')

In [10]:
# ===========================================
# OSRM Travel Time Calculation to ALL Ports
# ===========================================

# === OSRM Function ===
def osrm_route_time(lat1, lon1, lat2, lon2, retries=3):
    url = f"https://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    for _ in range(retries):
        try:
            r = requests.get(url, timeout=10)
            if r.status_code == 200:
                data = r.json()
                return data['routes'][0]['duration'] / 60 
        except Exception:
            pass
        time.sleep(1)
    return None  # if all attempts fail

# === Compute travel times ===
for _, port in puertos.iterrows():
    col_name = f"Tiempo_Prt_{port['portName']}"
    times = []
    print(f"Calculating {col_name} ...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        t = osrm_route_time(row['Latitud'], row['Longitud'], port['latitude'], port['longitude'])
        times.append(t)
        time.sleep(0.1)  # rate limiting
    df[col_name] = times

# === Save results ===
df.to_csv("Bases/minas_con_tiempos_puertos.csv", index=False)
print("✅ Travel times calculated and saved.")


Calculating Tiempo_Prt_Antofagasta ...


  0%|          | 18/15160 [00:18<4:20:25,  1.03s/it]


KeyboardInterrupt: 